# Prototyping

Creates a neural network which evaluates a time series and produce a set of predicted values for the time series
Predicted values may be used in a policy to make a trade. This policy may be modeled by simple multiple regression or a neural network.

## Data
Test data is taken as most recent to avoid lookahead bias. Train data is split into a validation and training set during fitting.


## TODO
- Convert feature percentages to stdev
- Adding VIX as a signal
- Adding High/Low as signals
- Multiple securities/ aggregate samples
- Policy network
- Regularization (l2)
- Dilated convolution

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import pandas_datareader as pdr
from datetime import datetime

def from_network(symbol):
    return pdr.get_data_yahoo(symbols=symbol, start=datetime(1900, 1, 1))

def from_file(symbol):
    dataset_path = keras.utils.get_file("{}.csv".format(symbol), "http://localhost:8000/data/daily/{}.csv".format(symbol))
    column_names = ['Date','Open','High','Low','Close','Adj Close','Volume'] 
    return pd.read_csv(dataset_path, 
                              names=column_names, 
                              dtype={'Close': np.float64,'Open': np.float64,'High': np.float64,'Adj Close': np.float64, 'Volume': np.float64},
                              header=0,
                              na_values = "?", 
                              comment='\t',
                              sep=",",
                              skipinitialspace=True)

#dataset = raw_dataset.copy()
dataset = from_network('SPY')
dataset = dataset.sort_values(by=['Date'],ascending=False)




In [2]:
print dataset

                  High         Low        Open       Close       Volume  \
Date                                                                      
2019-01-22  265.059998  261.059998  264.820007  262.859985  115355500.0   
2019-01-18  266.980011  263.000000  264.980011  266.459991  127900300.0   
2019-01-17  263.920013  259.959991  260.010010  262.959991   96118400.0   
2019-01-16  261.970001  260.600006  260.829987  260.980011   77636700.0   
2019-01-15  260.700012  257.809998  257.820007  260.350006   85208300.0   
2019-01-14  258.299988  256.410004  256.859985  257.399994   70908200.0   
2019-01-11  259.010010  257.029999  257.679993  258.980011   73858100.0   
2019-01-10  259.160004  255.500000  256.260010  258.880005   96823900.0   
2019-01-09  258.910004  256.190002  257.559998  257.970001   95006600.0   
2019-01-08  257.309998  254.000000  256.820007  256.769989  102512600.0   
2019-01-07  255.949997  251.690002  252.690002  254.380005  103139100.0   
2019-01-04  253.110001  2

In [43]:

dataset_stats = dataset.describe()
dataset_stats = dataset_stats.transpose()
NUM_INPUT_NEURONS = 64
NUM_OUTPUT_NEURONS = 3
NUM_SAMPLES = len(dataset)
NUM_TEST_SAMPLES = int(.33 * NUM_SAMPLES)
print NUM_SAMPLES
# Create features (only close price for now)
def convert_to_percentage(old, new):
    return (old - new) / old

def convert_labels_to_category(labels): 
    # Simplification - If positive return, 1, else 0
    # return map(lambda arr: 1 if arr[0] > 1 else 0, labels)
    # rounding simpliciation
    return map(lambda arr: map(lambda val: round(val,6),arr), labels)

def convert_to_train(raw_dataset):
    dataset = raw_dataset.copy()
    features = []
    labels = []
    for i in range(5, len(dataset) - NUM_INPUT_NEURONS):

        feature_dataset = dataset[i:i+NUM_INPUT_NEURONS].copy()
        latest_close = feature_dataset['Close'].iloc[0]
        
        features.append(
            feature_dataset['Close']
                .map(lambda current: convert_to_percentage(latest_close, current))
                .tolist()
        )
        labels.append([
            dataset['Close'].iloc[i-1] / latest_close, # 1 day trade
            dataset['Close'].iloc[i-3] / latest_close, # 3 day trade
            dataset['Close'].iloc[i-5] / latest_close, # 5 day trade
        ])
        
    # Without converting labels the precision is hard to determine accuracy. 
    # Rather than crude 0/1, maybe this can be more sophisticated
    labels = convert_labels_to_category(labels)
    
    return [features,labels]
converted_feature_set = convert_to_train(dataset)
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(converted_feature_set[0][0])]),
    layers.Dense(32, activation=tf.nn.relu),
    layers.Dense(16, activation=tf.nn.relu),
    layers.Dense(NUM_OUTPUT_NEURONS)
  ])

  optimizer = tf.train.RMSPropOptimizer(0.001)

  model.compile(loss='mse',
                optimizer='sgd',
                metrics=[
                    'mae',
                ])
  return model

tensorboard = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

model = build_model()
model.summary()

6542
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 51        
Total params: 6,819
Trainable params: 6,819
Non-trainable params: 0
_________________________________________________________________


In [44]:
#len(converted_feature_set[0][0])
print NUM_TEST_SAMPLES
train_data = np.array(converted_feature_set[0][NUM_TEST_SAMPLES:])
train_labels = np.array(converted_feature_set[1][NUM_TEST_SAMPLES:])

test_data = np.array(converted_feature_set[0][:NUM_TEST_SAMPLES])
test_labels = np.array(converted_feature_set[1][:NUM_TEST_SAMPLES])


2158


In [45]:
print train_data
print train_labels

[[ 0.          0.00160514  0.00124843 ... -0.03968251 -0.03415375
  -0.04369539]
 [ 0.         -0.00035728  0.02188279 ... -0.03581638 -0.04537336
  -0.04590925]
 [ 0.          0.02223212  0.02071428 ... -0.04500001 -0.0455357
  -0.03937503]
 ...
 [ 0.          0.00280309  0.00630583 ... -0.00840926 -0.00911114
  -0.00490651]
 [ 0.          0.00351259  0.00983838 ... -0.01194772 -0.00773127
   0.00281097]
 [ 0.          0.00634809  0.00775852 ... -0.0112835  -0.0007041
   0.00141044]]
[[0.996344 0.977082 0.95791 ]
 [1.001608 0.995088 0.975616]
 [0.999643 0.997589 0.978304]
 ...
 [0.996497 0.996497 0.9993  ]
 [1.002811 0.997189 1.003515]
 [1.003525 1.002821 1.002821]]


In [46]:

history = model.fit(
    train_data, train_labels,
    epochs=80, validation_split = 0.2, verbose=1,
    callbacks=[tensorboard]
)

Train on 3452 samples, validate on 863 samples
Epoch 1/80
3452/3452 [==============================] - 0s 80us/step - loss: 0.3399 - mean_absolute_error: 0.5157 - val_loss: 0.0291 - val_mean_absolute_error: 0.1523
Epoch 2/80
3452/3452 [==============================] - 0s 39us/step - loss: 0.0084 - mean_absolute_error: 0.0702 - val_loss: 6.4785e-04 - val_mean_absolute_error: 0.0199
Epoch 3/80
3452/3452 [==============================] - 0s 41us/step - loss: 0.0019 - mean_absolute_error: 0.0279 - val_loss: 2.7612e-04 - val_mean_absolute_error: 0.0131
Epoch 4/80
3452/3452 [==============================] - 0s 38us/step - loss: 0.0015 - mean_absolute_error: 0.0246 - val_loss: 2.2499e-04 - val_mean_absolute_error: 0.0118
Epoch 5/80
3452/3452 [==============================] - 0s 40us/step - loss: 0.0012 - mean_absolute_error: 0.0228 - val_loss: 1.9503e-04 - val_mean_absolute_error: 0.0109
Epoch 6/80
3452/3452 [==============================] - 0s 40us/step - loss: 0.0011 - mean_absolute_er

3452/3452 [==============================] - 0s 43us/step - loss: 6.0875e-04 - mean_absolute_error: 0.0173 - val_loss: 1.6449e-04 - val_mean_absolute_error: 0.0097
Epoch 48/80
3452/3452 [==============================] - 0s 42us/step - loss: 6.0754e-04 - mean_absolute_error: 0.0172 - val_loss: 1.6483e-04 - val_mean_absolute_error: 0.0097
Epoch 49/80
3452/3452 [==============================] - 0s 41us/step - loss: 6.0615e-04 - mean_absolute_error: 0.0172 - val_loss: 1.6397e-04 - val_mean_absolute_error: 0.0096
Epoch 50/80
3452/3452 [==============================] - 0s 40us/step - loss: 6.0490e-04 - mean_absolute_error: 0.0172 - val_loss: 1.6396e-04 - val_mean_absolute_error: 0.0096
Epoch 51/80
3452/3452 [==============================] - 0s 53us/step - loss: 6.0371e-04 - mean_absolute_error: 0.0172 - val_loss: 1.6362e-04 - val_mean_absolute_error: 0.0096
Epoch 52/80
3452/3452 [==============================] - 0s 44us/step - loss: 6.0231e-04 - mean_absolute_error: 0.0172 - val_loss: 1

In [47]:
outputs = model.predict(test_data)

In [48]:
print test_data[0]
print test_labels[0]
print outputs[0]

[ 0.         -0.00613837 -0.00574985 -0.00221448  0.00244757  0.01173267
  0.01946385  0.05124315  0.02804973  0.02905981  0.03749026  0.03624703
  0.04358975  0.08958818  0.06487955  0.03974357  0.02385392  0.00901318
  0.00792538 -0.01192699 -0.03096349 -0.03131312 -0.02614612 -0.02591303
 -0.02397053 -0.04832946 -0.04992232 -0.08508156 -0.07090132 -0.06441343
 -0.06674434 -0.04273504 -0.03923856 -0.0227273  -0.02960371 -0.02610723
 -0.04545459 -0.06344218 -0.06068374 -0.04972812 -0.05695417 -0.05893556
 -0.07909874 -0.08974362 -0.09172501 -0.06884228 -0.06212129 -0.05629379
 -0.06258748 -0.05139865 -0.04028747 -0.02509709 -0.03080805 -0.04926182
 -0.03076928 -0.06297588 -0.06841498 -0.07323235 -0.07381508 -0.08954941
 -0.08935509 -0.06604507 -0.07206689 -0.05738158]
[1.011461 1.021601 1.021212]
[0.9885347 1.0004399 1.0051986]


In [49]:
# one day return
actual = map(lambda arr: arr[0],test_labels)
# signal step for our policy network
signals = map(lambda arr: sum(arr) / len(arr),outputs)
# primitive policy temporarily in place of a RL policy network
trades = map(lambda signal: 1 if round(signal,2) > 1 else 0,signals)

In [50]:
df = pd.DataFrame({
    'signal':signals,
    'actual': actual,
    'trade':trades,
})

In [51]:
df['entry_success'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 1) else 0,axis=1)
df['entry_failure'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 1) else 0,axis=1)
df['avoid_success'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 0) else 0,axis=1)
df['avoid_failure'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 0) else 0,axis=1)


In [52]:
# primitive policy - replace with a policy network which maximizes reward
def label_success (row):
    return 0 if (row['entry_failure'] == 1 or row['entry_failure'] == 1) else 1

success = df.apply (lambda row: label_success (row),axis=1)
df['success'] = success;
df

,actual,signal,trade,entry_success,entry_failure,avoid_success,avoid_failure,success
0,1.011461,0.998058,0,0,0,0,1,1
1,0.993899,0.993029,0,0,0,1,0,1
2,1.000386,0.994800,0,0,0,0,1,1
3,1.003528,0.996154,0,0,0,0,1,1
4,1.004673,1.004147,0,0,0,0,1,1
5,1.009395,1.002490,0,0,0,0,1,1
6,1.007885,1.006909,1,1,0,0,0,1
7,1.033496,1.015064,1,1,0,0,0,1
8,0.976137,1.003767,0,0,0,1,0,1
9,1.001040,0.997482,0,0,0,0,1,1


In [53]:
print '\nNon-loss events'
print sum(df['success'])
print sum(df['success']) / (NUM_TEST_SAMPLES * 1.00)

print '\nLose trades'
print sum(df['entry_failure'])
print sum(df['entry_failure']) / (NUM_TEST_SAMPLES * 1.00)

print '\nWin trades'
print sum(df['entry_success'])
print sum(df['entry_success']) / (NUM_TEST_SAMPLES * 1.00)

print '\nMissed opportunities'
print sum(df['avoid_failure'])
print sum(df['avoid_failure']) / (NUM_TEST_SAMPLES * 1.00)

print '\nBullets dodged'
print sum(df['avoid_success'])
print sum(df['avoid_success']) / (NUM_TEST_SAMPLES * 1.00)


Non-loss events
1942
0.899907321594

Lose trades
216
0.100092678406

Win trades
249
0.115384615385

Missed opportunities
926
0.429101019462

Bullets dodged
757
0.35078776645


In [54]:
df1 = dataset.copy().head(NUM_TEST_SAMPLES)
df2 = df.copy()

In [55]:
pd.concat([df1.reset_index(),df2], axis=1).drop(['High','Low','Open','Volume','Adj Close'], axis=1)

,Date,Close,actual,signal,trade,entry_success,entry_failure,avoid_success,avoid_failure,success
0,2019-01-22,262.859985,1.011461,0.998058,0,0,0,0,1,1
1,2019-01-18,266.459991,0.993899,0.993029,0,0,0,1,0,1
2,2019-01-17,262.959991,1.000386,0.994800,0,0,0,0,1,1
3,2019-01-16,260.980011,1.003528,0.996154,0,0,0,0,1,1
4,2019-01-15,260.350006,1.004673,1.004147,0,0,0,0,1,1
5,2019-01-14,257.399994,1.009395,1.002490,0,0,0,0,1,1
6,2019-01-11,258.980011,1.007885,1.006909,1,1,0,0,0,1
7,2019-01-10,258.880005,1.033496,1.015064,1,1,0,0,0,1
8,2019-01-09,257.970001,0.976137,1.003767,0,0,0,1,0,1
9,2019-01-08,256.769989,1.001040,0.997482,0,0,0,0,1,1
